In [1]:
import numpy as np
from transonic import jit


def fxfy(ft, fn, theta):
    sin_theta = np.sin(theta)
    cos_theta = np.cos(theta)
    fx = cos_theta * ft - sin_theta * fn
    fy = sin_theta * ft + cos_theta * fn
    return fx, fy


def fxfy_loops(ft, fn, theta):
    n0 = theta.size
    fx = np.empty_like(ft)
    fy = np.empty_like(fn)
    for index in range(n0):
        sin_theta = np.sin(theta[index])
        cos_theta = np.cos(theta[index])
        fx[index] = cos_theta * ft[index] - sin_theta * fn[index]
        fy[index] = sin_theta * ft[index] + cos_theta * fn[index]
    return fx, fy


fxfy_pythran = jit(backend="pythran", native=True, xsimd=True)(fxfy)
fxfy_numba = jit(backend="numba")(fxfy)
fxfy_loops_pythran = jit(backend="pythran", native=True, xsimd=True)(fxfy_loops)
fxfy_loops_numba = jit(backend="numba")(fxfy_loops)

In [2]:
import numba
import pythran
from transonic import __version__, wait_for_all_extensions
from transonic.util import timeit

theta = np.linspace(0, 2 * np.pi, 10000)
ft = 2.5 * theta
fv = 1.5 * theta
out = fxfy(ft, fv, theta)
out_loops = fxfy_loops(ft, fv, theta)
assert np.allclose(out, out_loops)

print(
    f"transonic {__version__}\n"
    f"pythran {pythran.__version__}\n"
    f"numba {numba.__version__}\n"
)

loc = locals()

def bench(call, norm=None):
    ret = result = timeit(call, globals=loc)
    if norm is None:
        norm = result
    result /= norm
    print(f"{call.split('(')[0]:33s}: {result:.3f} * norm")
    return ret

# warmup
fxfy_pythran(ft, fv, theta)
fxfy_loops_pythran(ft, fv, theta)
fxfy_numba(ft, fv, theta)
fxfy_loops_numba(ft, fv, theta)

wait_for_all_extensions()

INFO     Schedule pythranization of file /home/pierre/.transonic/pythran/__jit__/__ipython__6ca57c6e93778ae96e3e4b88a3e893ab/fxfy.py
INFO     Schedule pythranization of file /home/pierre/.transonic/pythran/__jit__/__ipython__6ca57c6e93778ae96e3e4b88a3e893ab/fxfy_loops.py


transonic 0.4.1
pythran 0.9.3post1
numba 0.46.0

compile extension
compile extension


In [3]:
norm = bench("fxfy(ft, fv, theta)")
print(f"norm = {norm:.2e} s")

for backend in ("numba", "pythran"):
    bench(f"fxfy_{backend}(ft, fv, theta)", norm=norm)
    bench(f"fxfy_loops_{backend}(ft, fv, theta)", norm=norm)

fxfy                             : 1.000 * norm
norm = 2.68e-04 s
fxfy_numba                       : 0.956 * norm
fxfy_loops_numba                 : 0.917 * norm
fxfy_pythran                     : 0.217 * norm
fxfy_loops_pythran               : 0.918 * norm
